# Data pre-processing

__Goal:__ Build dataset for training and evaluation relation extraction models

__Method:__ Extract relation data from manually curated covid-19 kg, split into training and test sets

__Data:__ [Covid-19 KG](https://github.com/covid19kg/covid19kg) from Fraunhofer Institute for Algorithms and Scientific Computing SCAI

__Tools:__ PyBEL

__Result:__ covid-19-kg dataset

In [ ]:
import requests
import pybel

In [ ]:
#load graph
url = 'https://github.com/covid19kg/covid19kg/raw/master/covid19kg/_cache.bel.nodelink.json'
res = requests.get(url)
graph = pybel.from_nodelink(res.json())

In [ ]:
#export  graph as a tab-separated edge list.
pybel.to_csv(graph, 'covid19kg_dataset.csv')